In [249]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score as acc

In [250]:
outlook = ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain','Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast',np.NaN,'Rain', 'Sunny', 'Sunny']
temparature = [26, 28, 29, 23,np.NaN, 12,8, 25,18,20,20,21,26,24, 23,21]
humidity = ["High", "High", "High","High","Normal","Normal",np.NaN, "High", "Normal", "Normal", "Normal", "High", "Normal", "High", "Normal","Normal"]
wind = [np.NaN, "Strong", "Weak", "Weak", "Weak", "Strong", "Strong", "Weak", "Weak", "Weak", "Strong", "Strong", "Weak", "Strong", "Weak", "Weak"]
playTennis = ["No", "No", "Yes", "Yes", "Yes", "No", "Yes", "No", "Yes", "Yes",np.NAN, "Yes", "Yes", "No", "No", "Yes"]
day = ["D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", "D13", "D14", "D15", "D16"]

df =  pd.DataFrame(list(zip(day,outlook,temparature,humidity,wind,playTennis)), columns=['Day', 'Outlook', 'Temparatur','Humidity','Wind','PlayTennis'])
df

,Day,Outlook,Temparatur,Humidity,Wind,PlayTennis
0,D1,Sunny,26.0,High,NaN,No
1,D2,Sunny,28.0,High,Strong,No
2,D3,Overcast,29.0,High,Weak,Yes
3,D4,Rain,23.0,High,Weak,Yes
4,D5,Rain,NaN,Normal,Weak,Yes
5,D6,Rain,12.0,Normal,Strong,No
6,D7,Overcast,8.0,NaN,Strong,Yes
7,D8,Sunny,25.0,High,Weak,No
8,D9,Sunny,18.0,Normal,Weak,Yes
9,D10,Rain,20.0,Normal,Weak,Yes


In [251]:
df.dtypes

Day            object
Outlook        object
Temparatur    float64
Humidity       object
Wind           object
PlayTennis     object
dtype: object

In [252]:
# Finding number of missing values in each columns
df.isnull().sum()

Day           0
Outlook       1
Temparatur    1
Humidity      1
Wind          1
PlayTennis    1
dtype: int64

In [253]:
numeric_features=['Temparatur']
categorical_features = ['Outlook', 'Humidity', 'Wind', 'PlayTennis']

In [254]:
df = df.set_index('Day')

In [255]:
df_notna = df.dropna()
df_notna

,Outlook,Temparatur,Humidity,Wind,PlayTennis
Day,,,,,
D2,Sunny,28.0,High,Strong,No
D3,Overcast,29.0,High,Weak,Yes
D4,Rain,23.0,High,Weak,Yes
D6,Rain,12.0,Normal,Strong,No
D8,Sunny,25.0,High,Weak,No
D9,Sunny,18.0,Normal,Weak,Yes
D10,Rain,20.0,Normal,Weak,Yes
D12,Overcast,21.0,High,Strong,Yes
D14,Rain,24.0,High,Strong,No


In [256]:
df_numeric = df.copy()
for i in categorical_features:
    #df[i] = df[i].astype('category')
    #df[i] = df[i].cat.codes
    df_numeric[i] = pd.factorize(df_numeric[i])[0] # NaN to -1
df_numeric

,Outlook,Temparatur,Humidity,Wind,PlayTennis
Day,,,,,
D1,0,26.0,0,-1,0
D2,0,28.0,0,0,0
D3,1,29.0,0,1,1
D4,2,23.0,0,1,1
D5,2,NaN,1,1,1
D6,2,12.0,1,0,0
D7,1,8.0,-1,0,1
D8,0,25.0,0,1,0
D9,0,18.0,1,1,1


In [257]:
for i in categorical_features:
    df_numeric.loc[df_numeric[i] == -1, i] = np.NAN
df_numeric 

,Outlook,Temparatur,Humidity,Wind,PlayTennis
Day,,,,,
D1,0.0,26.0,0.0,NaN,0.0
D2,0.0,28.0,0.0,0.0,0.0
D3,1.0,29.0,0.0,1.0,1.0
D4,2.0,23.0,0.0,1.0,1.0
D5,2.0,NaN,1.0,1.0,1.0
D6,2.0,12.0,1.0,0.0,0.0
D7,1.0,8.0,NaN,0.0,1.0
D8,0.0,25.0,0.0,1.0,0.0
D9,0.0,18.0,1.0,1.0,1.0


In [258]:
#Normalize the features to range[0,1]
scaler = MinMaxScaler(feature_range=(0, 1))
df_knn = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
df_knn

,Outlook,Temparatur,Humidity,Wind,PlayTennis
0,0.0,0.857143,0.0,NaN,0.0
1,0.0,0.952381,0.0,0.0,0.0
2,0.5,1.000000,0.0,1.0,1.0
3,1.0,0.714286,0.0,1.0,1.0
4,1.0,NaN,1.0,1.0,1.0
5,1.0,0.190476,1.0,0.0,0.0
6,0.5,0.000000,NaN,0.0,1.0
7,0.0,0.809524,0.0,1.0,0.0
8,0.0,0.476190,1.0,1.0,1.0
9,1.0,0.571429,1.0,1.0,1.0


In [260]:
def knn_regressor(df_nominalized,df_notna_, feature):
    nan_row = df_nominalized[df_nominalized[feature].isna()]
    df_no_nona = df_nominalized.dropna()
    X = df_no_nona.drop([feature], axis=1)
    y = df_notna_[feature]
    
    if feature in numeric_features:
        knn = KNeighborsRegressor(n_neighbors=3, p=1)
        knn.fit(X, y)
        pred = knn.predict(X)
        #print("Accuracy of the model ist ", acc(y, pred))

        pred_feature = knn.predict(nan_row.drop([feature], axis=1))
        print(int(pred_feature))

knn_regressor(df_knn, df_notna, 'Temparatur')

20


In [261]:
def knn_calculate(df_nominalized,df_notna_, feature):
    nan_row = df_nominalized[df_nominalized[feature].isna()]
    df_no_nona = df_nominalized.dropna()
    X = df_no_nona.drop([feature], axis=1)
    y = df_notna_[feature]
    
    if feature in categorical_features:
        knn = KNeighborsClassifier(n_neighbors=3, p=1)
        knn.fit(X, y)
        pred = knn.predict(X)
        print("Accuracy of the model ist ", acc(y, pred))

        temp = nan_row.drop([feature], axis=1)
        # if temp.isna().values.any():
        #     for i in [11.0, 26.0]:
        #         temp = temp.fillna(i)
        #         pred_feature = knn.predict(temp)
        #         print(pred_feature)
        # else:
        pred_feature = knn.predict(temp)
        print(pred_feature)

for feat in categorical_features:
    knn_calculate(df_knn, df_notna, feature=feat)
 

Accuracy of the model ist  0.5454545454545454
['Sunny']
Accuracy of the model ist  0.8181818181818182
['Normal']
Accuracy of the model ist  0.6363636363636364
['Strong']
Accuracy of the model ist  0.6363636363636364
['Yes']


Die Klassifikationslabels sind für die Hervorsagen von Outlook, Temparatur und Humidity irrelevant, da keine Kausalitäten zwischen PlayTennis und anderen features vorliegen.

In [262]:
# Fill the table with predicted values
values = {'Outlook':'Sunny', 'Temparatur': 23, 'Humidity' : 'Normal', 'Wind' : 'Strong', 'PlayTennis' : 'Yes'}
df_filled = df.fillna(value=values)
df_filled

,Outlook,Temparatur,Humidity,Wind,PlayTennis
Day,,,,,
D1,Sunny,26.0,High,Strong,No
D2,Sunny,28.0,High,Strong,No
D3,Overcast,29.0,High,Weak,Yes
D4,Rain,23.0,High,Weak,Yes
D5,Rain,23.0,Normal,Weak,Yes
D6,Rain,12.0,Normal,Strong,No
D7,Overcast,8.0,Normal,Strong,Yes
D8,Sunny,25.0,High,Weak,No
D9,Sunny,18.0,Normal,Weak,Yes


In [263]:
#new_row = pd.DataFrame([['Sunny', 23, 'High', 'Strong', np.NaN]], columns = df_filled.columns, index=['D17'])
new_row = pd.DataFrame({'Outlook':'Sunny', 'Temparatur': 23, 'Humidity' : 'High', 'Wind' : 'Strong', 'PlayTennis' : np.NaN}, index = ['D17'])
df2 = pd.concat([df_filled, new_row])
df2

,Outlook,Temparatur,Humidity,Wind,PlayTennis
D1,Sunny,26.0,High,Strong,No
D2,Sunny,28.0,High,Strong,No
D3,Overcast,29.0,High,Weak,Yes
D4,Rain,23.0,High,Weak,Yes
D5,Rain,23.0,Normal,Weak,Yes
D6,Rain,12.0,Normal,Strong,No
D7,Overcast,8.0,Normal,Strong,Yes
D8,Sunny,25.0,High,Weak,No
D9,Sunny,18.0,Normal,Weak,Yes
D10,Rain,20.0,Normal,Weak,Yes


In [281]:
tmp = df2.iloc[-1,:]
tmp

Outlook        Sunny
Temparatur      23.0
Humidity        High
Wind          Strong
PlayTennis       NaN
Name: D17, dtype: object

In [297]:
def knn_predict(df_,feature,k):
    df_numeric_ = df_.copy()
    for i in categorical_features:
        df_numeric_[i] = pd.factorize(df_numeric_[i])[0]
    for i in categorical_features:
        df_numeric_.loc[df_numeric_[i] == -1, i] = np.NAN
    #print(df_numeric_)
    #Normalize the features to range[0,1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_knn_t = pd.DataFrame(scaler.fit_transform(df_numeric_), columns=df_.columns, index=df_.index)
    #print(df_knn_t)
    row_to_predict = pd.DataFrame(df_knn_t.loc['D17']).T
    #print(row_to_predict)
    y_train = df_.drop(['D17'])[feature]
    df_knn_t = df_knn_t.drop(['D17'], axis=0)
    X_train = df_knn_t.drop([feature], axis=1)
    
    knn = KNeighborsClassifier(n_neighbors= k, p=1)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_train)
    print("Accuracy of the model ist ", acc(y_train, pred))
    pred_feature = knn.predict(row_to_predict.drop([feature], axis=1))
    return pred_feature

i = 1
while True:
    result = knn_predict(df2,'PlayTennis',i)
    print(result)
    if result == ['Yes']:
        print(i)
        break
    i = i + 1



Accuracy of the model ist  1.0
['No']
Accuracy of the model ist  0.6875
['No']
Accuracy of the model ist  0.75
['No']
Accuracy of the model ist  0.8125
['No']
Accuracy of the model ist  0.75
['No']
Accuracy of the model ist  0.75
['No']
Accuracy of the model ist  0.875
['No']
Accuracy of the model ist  0.8125
['No']
Accuracy of the model ist  0.6875
['No']
Accuracy of the model ist  0.6875
['No']
Accuracy of the model ist  0.625
['Yes']
11
